# Set up environment for GPU (!important)

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import keras
import tensorflow as tf
print("TF version", tf.__version__)
print("keras version:", keras.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# If this last line give an error, stop the notebook kernel, reset it and run again

TF version 2.4.1
keras version: 2.4.3
Num GPUs Available:  1


# Train module

## Import required modules to process data

In [3]:
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

## Train configuration

In [4]:
IMAGE_SIZE      = 96    # Images are 96x96 px
IMAGE_CHANNELS  = 3     # Images are 3 chanell (RGB)

## Load train data info

In [5]:
# Function to append image file extension to train img ids
def appendExt(id):
    return id + ".tif"

# Load CSVs
traindf = pd.read_csv("/dataset/train_labels.csv")

# ========= FOR PROTOTYPING ONLY =========== #
# traindf = traindf[:100]
# ========================================== #

# Add extensions to id files
traindf["id"] = traindf["id"].apply(appendExt)

# Labels must be strings
traindf["label"] = traindf["label"].astype(str)

# removing this image because it caused a training error previously
traindf[traindf['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
traindf[traindf['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835.tif,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif,1
2,755db6279dae599ebb4d39a9123cce439965282d.tif,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif,0
4,068aba587a4950175d04c680d38943fd488d6a9d.tif,0
...,...,...
220020,53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6.tif,0
220021,d4b854fe38b07fe2831ad73892b3cec877689576.tif,1
220022,3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75.tif,0
220023,f129691c13433f66e1e0671ff1fe80944816f5a2.tif,0


## Build image data generator

In [6]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "/dataset/train/",
    x_col = "id",
    y_col = "label",
    subset = "training",
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = 10,
    shuffle = True,
    class_mode = "categorical",
)

valid_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "/dataset/train/",
    x_col = "id",
    y_col = "label",
    subset = "validation",
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = 10,
    shuffle = True,
    class_mode = "categorical",
)


Found 165019 validated image filenames belonging to 2 classes.
Found 55006 validated image filenames belonging to 2 classes.


In [7]:
# Calculate class weigths
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(traindf['label']),
    y=traindf['label']
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

Class weights: {0: 0.840380267057781, 1: 1.234472659537462}


## Build example model

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [9]:
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))


model.compile(
    optimizers.Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 90, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 45, 45, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 43, 43, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 41, 41, 64)        3

## Train the example model

In [10]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

print("STEP_SIZE_TRAIN:", STEP_SIZE_TRAIN)
print("STEP_SIZE_VALID:", STEP_SIZE_VALID)

# Save best model
checkpointPath = "/usr/src/scripts/best-model.h5"
checkpoint = ModelCheckpoint(
    checkpointPath,
    monitor='val_accuracy',
    verbose=1, 
    save_best_only=True,
    mode='max'
)

# Dynamic learning rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2, 
    verbose=1,
    mode='max',
    min_lr=0.00001
)
                                                                
callbacks_list = [checkpoint, reduce_lr]

STEP_SIZE_TRAIN: 16501
STEP_SIZE_VALID: 5500


In [11]:
model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    class_weight=class_weights,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=10, # Only for test!
    verbose=1,
    callbacks=callbacks_list
)


Epoch 1/10
16501/16501 [==============================] - 279s 17ms/step - loss: 0.4872 - accuracy: 0.7675 - val_loss: 0.5604 - val_accuracy: 0.7587

Epoch 00001: val_accuracy improved from -inf to 0.75871, saving model to /usr/src/scripts/best-model.h5
Epoch 2/10
16501/16501 [==============================] - 269s 16ms/step - loss: 0.3555 - accuracy: 0.8458 - val_loss: 0.3169 - val_accuracy: 0.8666

Epoch 00002: val_accuracy improved from 0.75871 to 0.86656, saving model to /usr/src/scripts/best-model.h5
Epoch 3/10
16501/16501 [==============================] - 257s 16ms/step - loss: 0.3053 - accuracy: 0.8721 - val_loss: 0.2665 - val_accuracy: 0.8878

Epoch 00003: val_accuracy improved from 0.86656 to 0.88782, saving model to /usr/src/scripts/best-model.h5
Epoch 4/10
16501/16501 [==============================] - 253s 15ms/step - loss: 0.2698 - accuracy: 0.8913 - val_loss: 0.2903 - val_accuracy: 0.8755

Epoch 00004: val_accuracy did not improve from 0.88782
Epoch 5/10
16501/16501 [===

In [12]:
val_loss, val_accuracy = model.evaluate(valid_generator, steps=STEP_SIZE_VALID)

5500/5500 [==============================] - 36s 7ms/step - loss: 0.1932 - accuracy: 0.9244


## Predict test data

In [13]:
# Load test data
testdf = pd.read_csv("/dataset/sample_submission.csv")
testdf["id"] = testdf["id"].apply(appendExt)

# Set up test data generator (only apply normalization)
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="/dataset/test/",
    x_col="id",
    y_col=None,
    batch_size=10,
    shuffle=False,
    class_mode=None,
    target_size=(IMAGE_SIZE,IMAGE_SIZE)
)


STEP_SIZE_TEST = test_generator.n//test_generator.batch_size


Found 57458 validated image filenames.


In [14]:
test_generator.reset()
model.predict(test_generator, steps=STEP_SIZE_TEST)

array([[0.9757699 , 0.02423015],
       [0.61150044, 0.38849956],
       [0.98754615, 0.01245388],
       ...,
       [0.11107685, 0.88892317],
       [0.72510606, 0.27489388],
       [0.99171174, 0.00828821]], dtype=float32)